In [1]:
module O

using OffsetArrays

mutable struct POA
    n::Int
    a::OffsetVector{Float64}
    A::OffsetArray{Float64, 2}
    x::OffsetVector{Float64}
    POA() = new()
end

function test_oa!(p::POA)
    range = 0:p.n-1
    a, A, x = p.a, p.A, p.x
    for i in range
        for j in range
            x[i] += A[j,i]*a[i]
        end
    end
end

p = POA()
p.n = 1000
p.a = OffsetVector{Float64}(rand(Float64,p.n), 0:p.n-1)
p.A = OffsetArray{Float64}(rand(Float64,p.n,p.n), 0:p.n-1, 0:p.n-1)
p.x = OffsetVector{Float64}(zeros(Float64, p.n), 0:p.n-1)

end

@time O.test_oa!(O.p)
@time O.test_oa!(O.p)

  0.269906 seconds (7.47 M allocations: 115.371 MiB, 23.24% gc time, 9.83% compilation time)
  0.165271 seconds (7.44 M allocations: 113.525 MiB, 13.30% gc time)


In [2]:
@code_warntype O.test_oa!(O.p)

MethodInstance for Main.O.test_oa!(::Main.O.POA)
  from test_oa!(p::Main.O.POA) @ Main.O In[1]:13
Arguments
  #self#::Core.Const(Main.O.test_oa!)
  p::Main.O.POA
Locals
  @_3::Union{Nothing, Tuple{Int64, Int64}}
  x::OffsetArrays.OffsetVector{Float64, AA} where AA<:AbstractVector{Float64}
  A::OffsetArrays.OffsetMatrix{Float64, AA} where AA<:AbstractMatrix{Float64}
  a::OffsetArrays.OffsetVector{Float64, AA} where AA<:AbstractVector{Float64}
  range::UnitRange{Int64}
  @_8::Union{Nothing, Tuple{Int64, Int64}}
  i::Int64
  j::Int64
Body::Nothing
1 ─ %1  = Base.getproperty(p, :n)::Int64
│   %2  = (%1 - 1)::Int64
│         (range = 0:%2)
│   %4  = Base.getproperty(p, :a)::OffsetArrays.OffsetVector{Float64, AA} where AA<:AbstractVector{Float64}
│   %5  = Base.getproperty(p, :A)::OffsetArrays.OffsetMatrix{Float64, AA} where AA<:AbstractMatrix{Float64}
│   %6  = Base.getproperty(p, :x)::OffsetArrays.OffsetVector{Float64, AA} where AA<:AbstractVector{Float64}
│         (a = %4)
│         (A =

In [3]:
module Q

using OffsetArrays

mutable struct POA{
        Ta<:AbstractVector,
        TA<:AbstractMatrix,
        Tx<:AbstractVector
    }
    a::Ta
    A::TA
    x::Tx
end

function test_oa!(p::POA)
    (; a, A, x) = p
    for i in axes(A, 2)
        for j in axes(A, 1)
            x[i] += A[j,i]*a[i]
        end
    end    
end

m = 10
n = 20
a = OffsetVector{Float64}(collect(1:n), 0:n-1)
A = OffsetMatrix{Float64}((1:m) * (1:n)', 0:m-1, 0:n-1)
x = OffsetVector{Float64}(zeros(n), 0:n-1)
p = POA(a, A, x)

end

@time Q.test_oa!(Q.p)
@time Q.test_oa!(Q.p)

  0.022013 seconds (23.54 k allocations: 1.606 MiB, 99.87% compilation time)
  0.000004 seconds


In [4]:
@code_warntype Q.test_oa!(Q.p)

MethodInstance for Main.Q.test_oa!(::Main.Q.POA{OffsetArrays.OffsetVector{Float64, Vector{Float64}}, OffsetArrays.OffsetMatrix{Float64, Matrix{Float64}}, OffsetArrays.OffsetVector{Float64, Vector{Float64}}})
  from test_oa!(p::Main.Q.POA) @ Main.Q In[3]:15
Arguments
  #self#::Core.Const(Main.Q.test_oa!)
  p::Main.Q.POA{OffsetArrays.OffsetVector{Float64, Vector{Float64}}, OffsetArrays.OffsetMatrix{Float64, Matrix{Float64}}, OffsetArrays.OffsetVector{Float64, Vector{Float64}}}
Locals
  @_3::Union{Nothing, Tuple{Int64, Int64}}
  x::OffsetArrays.OffsetVector{Float64, Vector{Float64}}
  A::OffsetArrays.OffsetMatrix{Float64, Matrix{Float64}}
  a::OffsetArrays.OffsetVector{Float64, Vector{Float64}}
  @_7::Union{Nothing, Tuple{Int64, Int64}}
  i::Int64
  j::Int64
Body::Nothing
1 ─       (a = Base.getproperty(p, :a))
│         (A = Base.getproperty(p, :A))
│         (x = Base.getproperty(p, :x))
│   %4  = Main.Q.axes(A, 2)::OffsetArrays.IdOffsetRange{Int64, Base.OneTo{Int64}}
│         (@_3 = B

In [5]:
q = Q.POA(Q.p.a.parent, Q.p.A.parent, zero(Q.p.x.parent))
@time Q.test_oa!(q)
@time Q.test_oa!(q)
q.x == Q.p.x.parent

  0.013624 seconds (7.42 k allocations: 509.461 KiB, 99.81% compilation time)
  0.000004 seconds


true